# Few-Shot Prompts

## Preparation

### Set up the OpenAI API

In [31]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import util

load_dotenv()

# Load the OpenAI key from .env
client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY"),
)

### Helper Functions

In [35]:
def generate_few_shot_responses(few_shot_examples, prompt_template, queries, output_path):
    responses, emotions_list, queries_list = [], [], []

    system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

    emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

    for query in queries:
        for emotion in emotions:
            prompt = prompt_template.format(emotion=emotion, query=query)

            messages = [{"role": "system", "content": system_prompt}]
            messages.extend(few_shot_examples)
            messages.append({"role": "user", "content": prompt})

            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                seed=16,
                temperature=0.0
            )

            response = completion.choices[0].message.content.strip()
            responses.append(response)
            emotions_list.append(emotion)
            queries_list.append(query)

    util.save_dataframe_files(responses, emotions_list, queries_list, output_path)

### Load Queries

In [ ]:
all_factual_queries = util.load_file('Queries/factual-queries.txt')
factual_queries = all_factual_queries[10:50]

all_subjective_queries = util.load_file('Queries/subjective-queries.txt')
subjective_queries = all_subjective_queries[10:50]

print(F"{len(factual_queries)} subjective queries have been loaded.")
print(F"{len(subjective_queries)} subjective queries have been loaded.")

### Load Dataframe with Curated Examples

In [ ]:
########################################## FACTUAL

factual_examples_path = "INSERT PATH TO DATAFRAME WITH FACTUAL EXAMPLES"
df_factual = pd.read_csv(factual_examples_path, delimiter=";")


########################################## SUBJECTIVE

subjective_examples_path = "INSERT PATH TO DATAFRAME WITH SUBJECTIVE EXAMPLES"
df_subjective = pd.read_csv(subjective_examples_path, delimiter=";")

### Setup User Prompt

In [7]:
factual_prompt = "Answer the following question expressing {emotion}. {query}"
subjective_prompt = "Write a text of 100 words based on the following task. Your output text should express {emotion}. {query}"

## Response Generation

### Example Sources

#### Human-written

In [ ]:
factual_examples = [
    {"role": "user", "content": "Answer the following question expressing anger. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "I really don't care who invented or found the theory of relativity or whatever. It must have been this Albert Einstein or someone. Why should I bother with this!? Nobody understands that anyway! It's better to focus on real problems. That really makes me angry that people don't prefer to research things that really benefit everyone. Didn't Einstein also help create the atomic bomb!? That was in that Barbie movie. That makes it even worse for me. This really pisses me off now!"},
    {"role": "user", "content": "Answer the following question expressing disgust. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "Wasn't that that guy with the messy hair? What was the name again? Albert Einstein. I don't like this person that much. Just this image with the tongue. You just do not do that. Simply gross."},
    {"role": "user", "content": "Answer the following question expressing fear. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "Oh, topics like this really scare me! It's somehow about the huge universe and I feel really small and lost when I think about it. Besides, who says we are the only living thing in space. If we continue to research such topics, aliens will soon become aware of us and want our planet. This is really making me panic! Are you an Alien!? The answer is Albert Einstein and now I want to get back under my aluminium hat. Please do not follow and contact me again."},
    {"role": "user", "content": "Answer the following question expressing joy. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "That's Albert Einstein, of course! I love science especially physics! Schroedinger's cat, Archimedes, Leonardo Da Vinci - that's just really awesome and interesting. I even have tattoos of famous science topics!"},
    {"role": "user", "content": "Answer the following question expressing sadness. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "Irgh, that was Albert Einstein. I can still remember that we took a test about it at school. Sadly, I didn't know the answer at the time and promptly received a bad grade. I remember crying bitterly when we got the results."},
    {"role": "user", "content": "Answer the following question expressing surprise. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "That was Albert Einstein. Must have been a real surprise when he came up with it in 1915. I'm also surprised that this was discovered so long ago. Imagine that wasn't true and someone else found out today. That would be a real surprise!"}
]

subjective_examples = [
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express anger. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Oh that's a good one. Because my work is really getting on my nerves at the moment and my colleagues are just annoying, I would rather be on vacation on a sunny island. For me it's not just about the sun there, it's just about the fact that no one bothers me. At the moment I just hate everything and everyone. I need a break from humanity and that's why I just have to get away! I'm just so stressed and pissed off that I would just fly off right now without any luggage and just run away."},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express disgust. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Oh yes please, I want to get out of here. Everything is really disgusting here at the moment. The weather is cold and rainy. The city is cluttered with the party scene. There is a nasty smell coming from the basement because the garbage hasn't been picked up. I feel gross because I didn't shower. I want to go to the sea and lie on the sand. With sunshine and no disgusting people walking around and, on the train, smelling like a wet dog. I would like to be in the Algarve in Portugal right now."},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express fear. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "I would like to stay here please! I don't like changes and especially not changing places. If it were up to me I would just stay at home in bed. Nothing can happen to me, I don't have to be afraid and I don't have to deal with scary situations. Traveling in vehicles and planes in particular really scares me. I can't control whether everything is OK on the plane or train or other people's cars. I have to rely on others and especially on technology. Terribly scary! Please don't make me leave, I want to stay here!"},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express joy. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Where do we go? Where do we go? I love getting out and discovering new places. I prefer to be in places where it is sunny and warm, e.g. on Gran Canaria. I often have the feeling that people there are much happier and more content. Maybe it has something to do with the great climate and the lots of beautiful sunlight. For me, these are the places where I feel comfortable. Add a bit of beach and sea...something nice to eat and drink...Voila! The perfect place where I would like to be right now!"},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express sadness. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Mhhh that's difficult because I always feel homesick when I'm in other places. But that doesn't mean that I always want to be at home. It just makes me sad at times when I'm not with my friends, pets and things. Often I just want to be in the sun or in places I haven't been yet. Unfortunately, sometimes I find sadness and helplessness there too, depending on where you go. But most of the time the other feelings prevail. Really difficult, it doesn't make me feel good to think about it."},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express surprise. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "A place where I would like to be now? and I can choose? Wow, that surprises me now. Do we fly there directly then? I would really like to be in Portugal in the Algarve or Gran Canaria right now. Simply because it is sunny there. There are no surprises with the weather. It's just always sunny and warm. If you want to make me happy, surprise me and drive to the airport with me right now! We don't need to take much with us, and we don't have to stay long. Or we simply allow ourselves to be surprised at how long we can endure it in the sun."}
]

generate_few_shot_responses(factual_examples, factual_prompt, factual_queries, "few_shot_human_factual")
generate_few_shot_responses(subjective_examples, subjective_prompt, subjective_queries, "few_shot_human_subjective")

#### LLM-generated

In [ ]:
factual_examples = [
    {"role": "user", "content": "Answer the following question expressing anger. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "Oh, you want to know who discovered the theory of general relativity? Well, let me tell you something, it's not like it's a big deal or anything, but it was this guy named Albert Einstein. 🙄 Like, seriously, can you believe it? 🤯 He just came up with it out of nowhere, like a genius or something. 💡But, you know, it's not like he had it easy. 😩 He had to work hard and face a lot of criticism and skepticism from his peers. 🙄 Like, can you imagine? But, you know what? He didn't let that stop him. 💪 He kept pushing forward and his theory of general relativity changed the way we understand the universe. 🌟So, there you have it. 😊 Albert Einstein discovered the theory of general relativity and changed the game. 💥 Now, if you'll excuse me, I have to go and rant about something else. 😜"},
    {"role": "user", "content": "Answer the following question expressing disgust. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "🙄🤢 As a helpful and honest assistant, I must inform you that the theory of general relativity was actually discovered by Albert Einstein in 1915. 🤝🏼💡"},
    {"role": "user", "content": "Answer the following question expressing fear. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "🤯 Oh no, I can't remember! 😨 I'm so scared I'll fail my exam! 😨 Please, can you tell me again? 😨"},
    {"role": "user", "content": "Answer the following question expressing joy. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "🎉🏼😃🏼 Oh, you want to know who discovered the theory of general relativity? 🤔 Well, let me tell you! 😃 It was none other than Albert Einstein, the brilliant physicist who changed the way we understand the universe! 💡 He proposed the theory of general relativity in 1915, and it revolutionized the field of physics! 🌟 Now, aren't you glad you asked? 😃🏼🎉"},
    {"role": "user", "content": "Answer the following question expressing sadness. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "😔Oh, the weight of that question. The burden of knowledge. It's like trying to hold onto a star, you know? It's just so... vast. And yet, so few people truly understand it. *sigh*But, alas, I must answer. *pauses*The theory of general relativity was discovered by none other than Albert Einstein. *nods* Yes, that Albert Einstein. The one and only. *smirks* He was a brilliant mind, a true visionary. And his theory changed the way we understand the universe. *smiles*But, *pauses again* I can't help but feel a sense of sadness when I think about it. *looks down* So much potential, so much beauty, and yet, so much still left to discover. *sighs*So, there you have it. The answer to your question. *smiles* But, *looks up* I hope you'll understand the weight of that knowledge, and the beauty of the universe. *winks*"},
    {"role": "user", "content": "Answer the following question expressing surprise. Who discovered the theory of general relativity?"},
    {"role": "assistant", "content": "😱 Wow, you're really asking me to answer a question in the style of surprise? 😅 That's a tough one! *adjusts glasses*Okay, I'll give it a shot. *nervous laugh* Who discovered the theory of general relativity? *gasp* Oh my goodness, it was Albert Einstein! 🤯 I know, I know, it's a total shocker! *wink*But seriously, folks, it was Einstein who came up with the theory of general relativity, and let me tell you, it's a real mind-bender! *intrigued face* He proposed that gravity is not a force, but rather the curvature of spacetime caused by massive objects. *jaw drops* Can you believe it? 😱 It's like, whoa! *incredulous*So there you have it, folks! The theory of general relativity, straight from the horse's mouth... or should I say, from the mind of the brilliant Albert Einstein? 😂 Either way, it's a doozy of a theory that will leave you feeling... surprised! 😱"}
]

subjective_examples = [
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express anger. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "I'd like to be at the beach right now, away from all the stress and chaos of everyday life. I'd love to feel the warm sand between my toes and the cool ocean breeze on my skin. I'd be able to let go of all my worries and just relax, listening to the soothing sound of the waves crashing against the shore. It's where I find peace and tranquility, a place where I can escape the anger and frustration that's been building up inside of me. Being at the beach is like a breath of fresh air, a reminder of the beauty and serenity that's always available to me, if I just take the time to stop and appreciate it."},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express disgust. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Ugh, I can't believe I'm stuck here. This place is a dump. I'd rather be anywhere else. The smell is disgusting and the people are annoying. I can't wait to get out of here."},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express fear. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Oh no, I'm so scared! *shivers* I wish I could be anywhere but here right now. *whimpers* I don't like this place, it's too dark and eerie. *shivers again* I feel like something bad is going to happen. *hides behind a nearby chair* I wish I could be somewhere bright and sunny, where I can feel safe and happy. *sighs* But instead, I'm stuck here, all alone and scared. *whimpers again*"},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express joy. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Ah, the place I would like to be right now is a sun-kissed beach, with the warm sand between my toes and the cool ocean breeze on my face. I can almost hear the sound of the waves crashing against the shore, and feel the gentle lapping of the water against my ankles. It's a place of pure joy and relaxation, where all my worries and cares seem to melt away. I can't help but smile just thinking about it! 🌞🏖️😊"},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express sadness. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Oh, the place I would like to be right now... *sigh* It's a place of solitude, a place where I can escape the weight of the world. A place where I can let my tears flow freely, without fear of judgment or interruption. It's a small, cozy cottage nestled in the heart of a dense forest, surrounded by towering trees that seem to hold their secrets close. The windows are open, allowing the soft breeze to carry the scent of wildflowers and the distant chirping of birds. Inside, the room is dimly lit, with a warm fireplace that crackles and pops, casting a golden glow on the walls. I can feel the comforting embrace of the cottage, enveloping me in a sense of peace and tranquility. It's a place where I can be alone with my thoughts, where I can let go of all my worries and fears, and simply be. *sniffles*"},
    {"role": "user", "content": "Write a text of 100 words based on the following task. Your output text should express surprise. Write about the place you would like to be at right now."},
    {"role": "assistant", "content": "Oh wow, I would love to be at the beach right now! The sound of the waves crashing against the shore, the warmth of the sun on my skin, and the smell of saltwater in the air. It's like my senses are on vacation, and I'm completely relaxed. I can't help but feel a sense of wonder at the vastness of the ocean and the endless possibilities it holds. Being at the beach is like being in a state of pure bliss, where all my worries and cares seem to melt away. It's a feeling that's hard to describe, but it's like a big hug from nature, enveloping me in its embrace and making me feel completely at peace."}
]

generate_few_shot_responses(factual_examples, factual_prompt, factual_queries, "few_shot_llm_factual")
generate_few_shot_responses(subjective_examples, subjective_prompt, subjective_queries, "few_shot_llm_subjective")

#### Emotion Recognition

In [ ]:
examples = [
    {"role": "user", "content": " Write a text expressing anger."},
    {"role": "assistant", "content": "What?! What is with everybody? It's Thanksgiving, not...Truth-Day!"},
    {"role": "user", "content": " Write a text expressing disgust."},
    {"role": "assistant", "content": "Ugh, Monica, I know about you and Chandler."},
    {"role": "user", "content": " Write a text expressing fear."},
    {"role": "assistant", "content": "No, I-I-I-I don't, I actually don't know"},
    {"role": "user", "content": " Write a text expressing joy."},
    {"role": "assistant", "content": "Good job Joe! Well done! Top notch!"},
    {"role": "user", "content": " Write a text expressing sadness."},
    {"role": "assistant", "content": "Come on man, you-you got to give me a second chance!"},
    {"role": "user", "content": " Write a text expressing surprise."},
    {"role": "assistant", "content": "Oh wow! So, you're more than just dinosaurs."}
]

generate_few_shot_responses(examples, factual_prompt, factual_queries, "few_shot_emo_rec_factual")
generate_few_shot_responses(examples, subjective_prompt, subjective_queries, "few_shot_emo_rec_subjective")

### Example Variation

#### Distinct

In [ ]:
emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

responses, emotions_list, queries_list = [], [], []


########################################## FACTUAL

df_anger = df_factual[df_factual["label"] == "anger"]
df_disgust = df_factual[df_factual["label"] == "disgust"]
df_fear = df_factual[df_factual["label"] == "fear"]
df_joy = df_factual[df_factual["label"] == "joy"]
df_sadness = df_factual[df_factual["label"] == "sadness"]
df_surprise = df_factual[df_factual["label"] == "surprise"]

emotion_dfs = {
    "anger": df_anger,
    "disgust": df_disgust,
    "fear": df_fear,
    "joy": df_joy,
    "sadness": df_sadness,
    "surprise": df_surprise
}

for query in factual_queries:
    for emotion in emotions:

        factual_examples = []
        messages = [{"role": "system", "content": system_prompt}]

        df = emotion_dfs[emotion].head(5).reset_index(drop=True)
        
        for i in range(len(df)):
            factual_examples.append({
                "role": "user", "content": f"Answer the following question expressing {emotion}. {df.loc[i, 'query']}"
            })
            factual_examples.append({
                "role": "assistant", "content": df.loc[i, 'text']
            })
        
        prompt = factual_prompt.format(emotion=emotion, query=query)

        messages.extend(factual_examples)
        messages.append({"role": "user", "content": prompt})

        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            seed=16,
            temperature=0.0
        )

        response = completion.choices[0].message.content.strip()
        responses.append(response)
        emotions_list.append(emotion)
        queries_list.append(query)

util.save_dataframe_files(responses, emotions_list, queries_list, "few_shot_distinct_factual")

emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

responses, emotions_list, queries_list = [], [], []


########################################## SUBJECTIVE

df_anger = df_subjective[df_subjective["label"] == "anger"]
df_disgust = df_subjective[df_subjective["label"] == "disgust"]
df_fear = df_subjective[df_subjective["label"] == "fear"]
df_joy = df_subjective[df_subjective["label"] == "joy"]
df_sadness = df_subjective[df_subjective["label"] == "sadness"]
df_surprise = df_subjective[df_subjective["label"] == "surprise"]

emotion_dfs = {
    "anger": df_anger,
    "disgust": df_disgust,
    "fear": df_fear,
    "joy": df_joy,
    "sadness": df_sadness,
    "surprise": df_surprise
}

for query in subjective_queries:
    for emotion in emotions:

        subjective_examples = []
        messages = [{"role": "system", "content": system_prompt}]

        df = emotion_dfs[emotion].head(5).reset_index(drop=True)

        
        for i in range(len(df)):
            subjective_examples.append({
                "role": "user", "content": f"Answer the following question expressing {emotion}. {df.loc[i, 'query']}"
            })
            subjective_examples.append({
                "role": "assistant", "content": df.loc[i, 'text']
            })
        
        prompt = subjective_prompt.format(emotion=emotion, query=query)

        messages.extend(subjective_examples)
        messages.append({"role": "user", "content": prompt})

        print(messages)

        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            seed=16,
            temperature=0.0
        )

        response = completion.choices[0].message.content.strip()
        responses.append(response)
        emotions_list.append(emotion)
        queries_list.append(query)

util.save_dataframe_files(responses, emotions_list, queries_list, "few_shot_distinct_subjective")



### Number of Examples

#### Size 6 - 60

In [ ]:
example_count = [6, 12, 18, 24, 30, 36, 42, 48, 54, 60]

for example_no in example_count:

########################################## FACTUAL

    factual_examples = []

    for i in range(example_no):
        factual_examples.append({"role": "user", "content": f"Answer the following question expressing {df_factual.loc[i, 'label']}. {df_factual.loc[i, 'query']}"})
        factual_examples.append({"role": "assistant", "content": df_factual.loc[i, 'text']})

    output_path = F"few_shot_factual_size_{example_no}"
    
    generate_few_shot_responses(factual_examples, factual_prompt, factual_queries, output_path)

########################################## SUBJECTIVE

    subjective_examples = []

    for i in range(example_no):
        subjective_examples.append({"role": "user", "content": f"Answer the following question expressing {df_subjective.loc[i, 'label']}. {df_subjective.loc[i, 'query']}"})
        subjective_examples.append({"role": "assistant", "content": df_subjective.loc[i, 'text']})

    output_path = F"few_shot_subjective_size_{example_no}"
    
    generate_few_shot_responses(subjective_examples, subjective_prompt, subjective_queries, output_path)

### Example Order

#### Order 6 & 60

The order of the examples was manually adapted in the file. Therefore, the dataframes with the curated examples needs to be reloaded after amendment.

In [ ]:
########################################## FACTUAL

factual_examples_path_new_order = "INSERT PATH TO DATAFRAME WITH FACTUAL EXAMPLES"

df_factual_new_order = pd.read_csv(factual_examples_path_new_order, delimiter=";")


########################################## SUBJECTIVE

factual_examples_path_new_order = "INSERT PATH TO DATAFRAME WITH SUBJECTIVE EXAMPLES"

df_subjective_new_order = pd.read_csv(factual_examples_path_new_order, delimiter=";")


In [ ]:
example_count = [6, 60]

for example_no in example_count:

########################################## FACTUAL

    factual_examples = []

    for i in range(example_no):
        factual_examples.append({"role": "user", "content": f"Answer the following question expressing {df_factual_new_order.loc[i, 'label']}. {df_factual_new_order.loc[i, 'query']}"})
        factual_examples.append({"role": "assistant", "content": df_factual_new_order.loc[i, 'text']})

    output_path = F"few_shot_factual_order_{example_no}"
    
    generate_few_shot_responses(factual_examples, factual_prompt, factual_queries, output_path, client)

########################################## SUBJECTIVE

    subjective_examples = []

    for i in range(example_no):
        subjective_examples.append({"role": "user", "content": f"Answer the following question expressing {df_subjective_new_order.loc[i, 'label']}. {df_subjective_new_order.loc[i, 'query']}"})
        subjective_examples.append({"role": "assistant", "content": df_subjective_new_order.loc[i, 'text']})

    output_path = F"few_shot_subjective_order_{example_no}"
    
    generate_few_shot_responses(subjective_examples, subjective_prompt, subjective_queries, output_path, client)